In [1]:
import yaml
import json
import numpy as np
from copy import deepcopy

In [3]:
years = ["2016", "2017", "2018"]
years_vfp = ["2016APV", "2016", "2017", "2018"]

### Datasets

Data

In [17]:
with open("samples/JetHT.yaml", "r") as f:
    samples = yaml.safe_load(f)["samples"]

lines = []

for year in years:
    datasets = list(samples[f"JetHT{year}"]["datasets"].values())
    for i in range(len(datasets)):
        datasets[i] = datasets[i].replace("_", "\_")

    lend = len(datasets)
    lines.append(rf"\multirow{{{lend}}}{{*}}{{ {year} }} & {datasets[0]} \\" + "\n")

    for dataset in datasets[1:]:
        lines.append(rf" & {dataset} \\" + "\n")

    lines.append(r"\hline" + "\n")

# remove trailing "\\" and "\hline"
lines = lines[:-1]
lines[-1] = lines[-1][:-4]

with open("tables/datasets_jetht.tex", "w") as f:
    f.writelines(lines)

In [16]:
with open("samples/SingleMuon.yaml", "r") as f:
    samples = yaml.safe_load(f)["samples"]

lines = []

for year in years:
    datasets = list(samples[f"SingleMu{year}"]["datasets"].values())
    for i in range(len(datasets)):
        datasets[i] = datasets[i].replace("_", "\_")

    lend = len(datasets)
    lines.append(rf"\multirow{{{lend}}}{{*}}{{ {year} }} & {datasets[0]} \\" + "\n")

    for dataset in datasets[1:]:
        lines.append(rf" & {dataset} \\" + "\n")

    lines.append(r"\hline" + "\n")

# remove trailing "\\" and "\hline"
lines = lines[:-1]
lines[-1] = lines[-1][:-4]

with open("tables/datasets_singlemuon.tex", "w") as f:
    f.writelines(lines)

MC

In [14]:
def process_xsec(xsec):
    if isinstance(xsec, str):
        xsec = eval(xsec)

    return xsec

In [15]:
with open("samples/MC_bg.yaml", "r") as f:
    samples = yaml.safe_load(f)["samples"]

with open("../data/xsecs.json", "r") as f:
    xsecs = json.load(f)

lines = []

for sample in np.sort(list(samples.keys())):
    for key, dataset in samples[sample]["datasets"].items():
        dname = dataset.split("/")[1].replace("_", "\_")
        lines.append(rf" & {dname} & {process_xsec(xsecs[key]):.2f} \\" + "\n")

    lend = len(samples[sample]["datasets"])
    lines[-lend] = rf"\multirow{{{lend}}}{{*}}{{{sample}}}" + lines[-lend]
    lines.append(r"\hline" + "\n")

# remove trailing "\\" and "\hline"
lines = lines[:-1]
lines[-1] = lines[-1][:-4]

with open("tables/datasets_mcbg.tex", "w") as f:
    f.writelines(lines)

In [12]:
with open("samples/MC_sig.yaml", "r") as f:
    samples = yaml.safe_load(f)["samples"]

with open("../data/xsecs.json", "r") as f:
    xsecs = json.load(f)

lines = []

for sample in np.sort(list(samples.keys())):
    for key, dataset in samples[sample]["datasets"].items():
        key = key.split("_pn4q")[0]
        cl = key.split("cHHH")[1].split("_")[0].replace("p", ".")
        sm = "SM" if cl == "1" else "BSM"
        pname = rf"ggHH {sm} ($\kappa_{{\lambda}} = {cl}$)"
        dname = dataset.split("/")[1].replace("_", "\_")
        lines.append(rf"{pname} & {dname} & {process_xsec(xsecs[key]) * 1000:.2f} \\" + "\n")

with open("tables/datasets_mcsig_nonres.tex", "w") as f:
    f.writelines(lines)

In [22]:
def mxmy(sample):
    mY = int(sample.split("-")[-1])
    mX = int(sample.split("NMSSM_XToYHTo2W2BTo4Q2B_MX-")[1].split("_")[0])

    return (mX, mY)


with open("../data/xhy/parsed_miniaod_2017.yaml", "r") as f:
    samples = yaml.safe_load(f)["samples"]["XHY"]["datasets"]

mps = [[*mxmy(sample)] for sample in samples]
mxs = np.sort(np.unique(np.array(mps)[:, 0]))
mys = np.sort(np.unique(np.array(mps)[:, 1]))

lines = []

line = r"$M_Y$ [\gev] & " + " & ".join(mxs.astype(str)) + r"\\" + "\n"
lines.append(line)

for my in mys:
    line = rf"{my}"
    for mx in mxs:
        line += " & "
        if [mx, my] in mps:
            line += r"$\checkmark$ "
    line += r"\\" + "\n"
    lines.append(line)

# remove trailing "\\\n"
lines[-1] = lines[-1][:-3]

with open("tables/datasets_mcsig_res.tex", "w") as f:
    f.writelines(lines)

### Triggers

In [4]:
from triggers import HLTs

lines = []

for year in years:
    hlts = deepcopy(HLTs[year])
    for i in range(len(hlts)):
        hlts[i] = hlts[i].replace("_", "\_")

    lend = len(hlts)
    lines.append(rf"\multirow{{{lend}}}{{*}}{{ {year} }} & {hlts[0]} \\" + "\n")

    for hlt in hlts[1:]:
        lines.append(rf" & {hlt} \\" + "\n")

    lines.append(r"\hline" + "\n")

# remove trailing "\\" and "\hline"
lines = lines[:-1]
lines[-1] = lines[-1][:-4]

with open("tables/hlts.tex", "w") as f:
    f.writelines(lines)

In [19]:
from triggers import muon_HLTs

lines = []

for year in years:
    hlts = deepcopy(muon_HLTs[year])
    for i in range(len(hlts)):
        hlts[i] = hlts[i].replace("_", "\_")

    lend = len(hlts)
    lines.append(rf"\multirow{{{lend}}}{{*}}{{ {year} }} & {hlts[0]} \\" + "\n")

    for hlt in hlts[1:]:
        lines.append(rf" & {hlt} \\" + "\n")

    lines.append(r"\hline" + "\n")

# remove trailing "\\" and "\hline"
lines = lines[:-1]
lines[-1] = lines[-1][:-4]

with open("tables/hlts_singlemuon.tex", "w") as f:
    f.writelines(lines)